# SQL Agent from Scratch

In [1]:
import re
import os
import json
import duckdb
import numpy as np
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
from dotenv import load_dotenv

load_dotenv("../../.env")

True

## Data Preparartion

- Developing AI agent is also data engineering task. We need the data in order to provide context to LLM. In this project, we will use chinook, a sample database avilable in many databases, for demos and testing our sql agent.
- We obtained the data in csv format. To prepare database with minimum effort, we will use duckdb to store and retrieve the data. Below code is python script use for data preparation.
- **Reference**: https://www.sqlitetutorial.net/sqlite-sample-database/

In [ ]:
DATA_PATH = "../data"

# Create database
conn = duckdb.connect("../output/chinook.duckdb")

# Loop through csv files and create table in duckdb
for csv_file in os.listdir(DATA_PATH):
    # Split and exclude file extension
    table_name = csv_file.split(".")[0]
    conn.sql(f"""
        CREATE OR REPLACE TABLE {table_name} AS 
        SELECT * FROM read_csv('../data/{csv_file}')
    """)

# Verify table creation
conn.sql("SHOW TABLES")
conn.close()

In [3]:
def execute_sql(sql):
    # connect to chinook database
    conn = duckdb.connect("../output/chinook.duckdb")
    df = conn.sql(sql).df()

    return df

## Develop Agent

In [4]:
conn = duckdb.connect("../output/chinook.duckdb")
table_list = conn.sql("SHOW TABLES").df()["name"].to_list()

relationship = {file: [] for file in table_list}

# Set relationship of albums table
relationship["albums"] = {
    "AlbumId": "PRIMARY KEY", 
    "ArtistId": "FOREIGN KEY -> artists.ArtistId"
}

# Set relationship of artists table
relationship["artists"] = {
    "ArtistId": "PRIMARY KEY"
}

# Set relationship of customers table
relationship["customers"] = {
    "CustomerId": "PRIMARY KEY", 
    "SupportRepId": "FOREIGN KEY -> employees.EmployeeId"
}

# Set relationship of employees table
relationship["employees"] = {
    "EmployeeId": "PRIMARY KEY",
    "ReportsTo": "FOREIGN KEY -> employees.EmployeeId"
}

# Set relationship of genres table
relationship["genres"] = {
    "GenreId": "PRIMARY KEY"
}

# Set relationship of invoices table
relationship["invoices"] = {
    "InvoiceId": "PRIMARY KEY",
    "CustomerId": "FOREIGN KEY -> customers.CustomerId"
}

# Set relationship of invoice_items table
relationship["invoice_items"] = {
    "InvoiceItemId": "PRIMARY KEY",
    "InvoiceId": "FOREIGN KEY -> invoices.InvoiceId",
    "TrackId": "FOREIGN KEY -> tracks.TrackId"
}

# Set relationship of media types table
relationship["media_types"] = {
    "MediaTypeId": "PRIMARY"
}

# Set relationship of playlists table
relationship["playlists"] = {
    "PlaylistId": "PRIMARY KEY"
}

# Set relationship of playlist_track table
relationship["playlist_track"] = {
    "PlaylistId": "PRIMARY KEY", 
    "TrackId": "FOREIGN KEY -> tracks.TrackId"
}

# Set relationship of tracks table
relationship["tracks"] = {
    "TrackId": "PRIMARY KEY", 
    "MediaTypeId": "FOREIGN KEY -> media_types.MediaTypeId",
    "GenreId": "FOREIGN KEY -> genres.GenreId",
    "AlbumId": "FOREIGN KEY -> albums.AlbumId"
}

In [5]:
schema_dict = {}

for table in table_list:
    schema_df = conn.sql(f"DESCRIBE {table}").to_df()
    table_name = table
    column_list = []
    
    for _, row in schema_df.iterrows():
        if row["column_name"] in relationship[table]:
            column_list.append(f"{row['column_name']} ({row['column_type']}, {relationship[table][row['column_name']]})")
        else:
            column_list.append(f"{row['column_name']} ({row['column_type']})")

    col_info = "\n".join(column_list)
    schema_dict[table_name] = f"Table: {table_name}\n{col_info}\n"

with open("../output/chinook_schema.json", "w") as f:
    json.dump(schema_dict, f, indent = 4)


### Text2SQL 

In [ ]:
# Import table schema
file = open("../output/chinook_schema.json", "r")
schema_dict = json.load(file)

# Extract table schema to string format
schema_string = "\n".join([schema for schema in schema_dict.values()])

# Write system prompt to instruct LLM for SLQ generation task
system_prompt = f"""
You are expert in query generator. \
Your task is to write correct and optimized queries based on the database schema and user questions. \
    Use database schema delimited by tiple single quotes to consider the tables, columns and SQL generation.\
'''{schema_string}'''

Rules:
- Always use the column and table names exactly as defined in schema.\
- Return only SQL code (no explanation or markdown unless requested)\
- If multiple tables are required, use proper JOINs based on foreign key relationships.\
- If aggregation is needed, include GROUP BY as required.\
- If user input is ambiguous, write your best interpretation and comment it.\
- If the user provides question with term 'Sales Agent', keep in mind that Sales Agents are employees with Title 'Sales Support Agents'\
"""

In [11]:
def text2sql(question):
    client = OpenAI(
        api_key = os.getenv("TYPHOON_API_KEY"),
        base_url = os.getenv("TYPHOON_BASE_URL")
    )

    response = client.chat.completions.create(
        model = os.getenv("TYPHOON_MODEL"),
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ],
        max_tokens = 1024
    )

    return response.choices[0].message.content

print(text2sql("How many customers?"))

SELECT COUNT(*) AS customer_count
FROM customers;


## Develop Tool